# Lagrangian particle demonstration in a periodic domain.

**Particles are simulated in the Southern Ocean AVISO altimetrey velocity field.**

Author: Wenrui Jiang, Tom Haine Feb '23

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

import seaduck as sd

### Loading data

The AVISO velocity field dataset is a lat-lon grid with staggered velocity. It's a snapshot from a single time (see below). `seaduck.utils` provides a few datasets for testing and demonstration purposes. The corresponding dataset are called when the corresponding functions are called.

<div class="alert alert-block alert-warning">
<b>Download:</b> If you are running this notebook for the first time, The dataset needs to be downloaded and cached, which can be a little slow.
</div>

In [ ]:
ds = sd.utils.get_dataset("aviso")
ds

Now, create the `OceData` object which provides methods to translate between lat-lon and grid-oriented coords. The model coords frequently needed are also cached. 

In [ ]:
bathtub = sd.OceData(ds)

The object also contains a `tp` (topology) object. It looks at the shape of the coordinate data. In this case, based on longitude range, it thinks the type is `x_periodic`, which is correct for the Southern Ocean.

In [ ]:
bathtub.tp.typ

We've prepared the bathtub that the ducks are going to swim (passively drift) in. Now decide where and when to drop them. 

We are going to use a short-hand defined in `seaduck.utils` to define the initial position for simplicity. The returns are normal than four 1D `numpy` array of `lon, lat, dep`, and `time`. It is as simple as that.

In [ ]:
# Define the extend of the box
west = -180.0
east = 180.0
south = -75.0
north = -40.0
shallow = -10.0
deep = -10.0

time = "1970-01-01"

Nlon = 300  # How many along longitudinal direction?
Nlat = 30  # How many along latitudinal direction?
Ndep = 1  # How many along vertical direction?

x, y, z, t = sd.utils.easy_3d_cube(
    (west, east, Nlon),
    (south, north, Nlat),
    (shallow, deep, Ndep),
    time,
    print_total_number=True,
)

**Here is where the particles start on the map:**

In [ ]:
plt.figure(figsize=(15, 6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=170.0))
ax.plot(x, y, "r+", markersize=1, transform=ccrs.PlateCarree())
ax.coastlines()
plt.show()

The AVISO velocity data only has the horizontal velocity component. Or in other words, all the particles are assume to be at the same (implicit) level, which is the surface for this example. `seaduck` takes care of that by just setting:

In [ ]:
z = None

The AVISO velocity field is a snapshot and does not have a time dimension. Therefore, the time is only valid in a relative sense, i.e. how long has the simulation gone in "real" time.

The standard format for time in `seaduck` is seconds since `1970-01-01 00:00` and could be negative. Remember what time is set as the initial time?

In [ ]:
all(t == 0)

Let's now define the duration of the simulation (end before start means integrate backwards in time). We can again use a short hand from `seaduck.utils`.

In [ ]:
tf = sd.utils.convert_time("1970-02-01")

This is equivalent to:

We are only interested when the particle is still in the domain, we can disregard the ones left from the northern boundary by defining:

In [ ]:
def interested_in(p):
    return np.logical_and(-74.5 < p.lat, p.lat < -45.5)

### Now create the `OceInterp.lagrangian.particle` object.

We have all the information we need. The final step of preparation is to create the `seaduck.lagrangian.particle` object. We need to tell the `seaduck` where, when, which bathtub, as well as the name of the velocity components, because they are not great at guessing.

In [ ]:
p = sd.particle(
    x=x,
    y=y,
    z=z,
    t=t,
    data=bathtub,
    uname="u",
    vname="v",
    wname=None,
    callback=interested_in,
)
p

### Perform the particle trajectory simulation.

Notice how we define when to dump output (`normal_stops`) and when to update the velocity field (`update_stops`). By default, the `stops` returned by the integration is the combination (union) of `normal_stops` and `update_stops`. `raw` is a list of `OceInterp.eulerian.position` objects with the same length as `stops`.

In [ ]:
normal_stops = np.linspace(t[0], tf, 10)
stops, raw = p.to_list_of_time(normal_stops=normal_stops, update_stops=[])

### Extract the information from the position objects.

In [ ]:
lons = np.array([pt.lon for pt in raw]).T
lats = np.array([pt.lat for pt in raw]).T

Some particles passed through the dateline (periodic boundary). Here is some pre-processing to handle the plot.

In [ ]:
for i in range(len(lons)):
    diff = np.diff(lons[i])

    if max(abs(diff)) > 330:
        for j in np.where(abs(diff) > 300)[0]:
            j = int(j)
            lons[i, j + 1 :] -= np.sign(diff[j]) * 360

## Here are the trajectories:

In [ ]:
plt.figure(figsize=(24, 6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=170.0))
ax.plot(lons.T, lats.T, "darkblue", lw=0.3, transform=ccrs.PlateCarree())
ax.coastlines()